# Databricks ML Quickstart: Model Training

This notebook provides a quick overview of machine learning model training on Databricks. To train models, you can use libraries like scikit-learn that are preinstalled on the Databricks Runtime for Machine Learning. In addition, you can use MLflow to track the trained models, and Hyperopt with SparkTrials to scale hyperparameter tuning.

This tutorial covers:
- Part 1: Training a simple classification model with MLflow tracking
- Part 2: Hyperparameter tuning a better performing model with Hyperopt

For more details on productionizing machine learning on Databricks including model lifecycle management and model inference, see the ML End to End Example ([AWS](https://docs.databricks.com/applications/mlflow/end-to-end-example.html)|[Azure](https://docs.microsoft.com/azure/databricks/applications/mlflow/end-to-end-example)|[GCP](https://docs.gcp.databricks.com/applications/mlflow/end-to-end-example.html)).

### Requirements
- Cluster running Databricks Runtime 7.5 ML or above

### Libraries
Import the necessary libraries. These libraries are preinstalled on Databricks Runtime for Machine Learning ([AWS](https://docs.databricks.com/runtime/mlruntime.html)|[Azure](https://docs.microsoft.com/azure/databricks/runtime/mlruntime)|[GCP](https://docs.gcp.databricks.com/runtime/mlruntime.html)) clusters and are tuned for compatibility and performance.

In [0]:
%sql
create schema fiep;


In [0]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/maxreis86/FIEP-Machine-Learning-e-Computacao-em-Nuvem/main/aula_03_hr_analytics_keras/HR_Analytics/dataprep_df.csv')

In [0]:
sdf = spark.createDataFrame(df)
sdf.printSchema()

root
 |-- department: string (nullable = true)
 |-- region: string (nullable = true)
 |-- education: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- recruitment_channel: string (nullable = true)
 |-- kpis_met_80: long (nullable = true)
 |-- no_of_trainings: double (nullable = true)
 |-- previous_year_rating: double (nullable = true)
 |-- age: double (nullable = true)
 |-- length_of_service: double (nullable = true)
 |-- avg_training_score: double (nullable = true)
 |-- employee_id: long (nullable = true)
 |-- reference_date: string (nullable = true)
 |-- is_promoted: long (nullable = true)
 |-- dataset: string (nullable = true)



In [0]:
sdf.write.mode("overwrite").saveAsTable('fiep.dataprep_df')

In [0]:
%sql
select * from fiep.dataprep_df limit 20;

department,region,education,gender,recruitment_channel,kpis_met_80,no_of_trainings,previous_year_rating,age,length_of_service,avg_training_score,employee_id,reference_date,is_promoted,dataset
Sales & Marketing,region_7,Master's & above,f,sourcing,1,1.0,5.0,35.0,8.0,49.0,65438,2022-03,0,train
Operations,region_22,Bachelor's,m,other,0,1.0,5.0,30.0,4.0,60.0,65141,2022-02,0,train
Sales & Marketing,region_19,Bachelor's,m,sourcing,0,1.0,3.0,34.0,7.0,50.0,7513,2022-05,0,train
Sales & Marketing,region_23,Bachelor's,m,other,0,2.0,1.0,39.0,10.0,50.0,2542,2022-02,0,train
Technology,region_26,Bachelor's,m,other,0,1.0,3.0,45.0,2.0,73.0,48945,2022-08,0,test
Analytics,region_2,Bachelor's,m,sourcing,0,2.0,3.0,31.0,7.0,85.0,58896,2022-08,0,test
Operations,region_20,Bachelor's,f,other,0,1.0,3.0,31.0,5.0,59.0,20379,2022-08,0,test
Operations,region_34,Master's & above,m,sourcing,0,1.0,3.0,33.0,6.0,63.0,16290,2022-07,0,train
Analytics,region_20,Bachelor's,m,other,0,1.0,4.0,28.0,5.0,83.0,73202,2022-04,0,train
Sales & Marketing,region_1,Master's & above,m,sourcing,1,1.0,5.0,32.0,5.0,54.0,28911,2022-02,0,train


In [0]:
%sql
DESCRIBE TABLE FORMATTED fiep.dataprep_df

col_name,data_type,comment
department,string,null
region,string,null
education,string,null
gender,string,null
recruitment_channel,string,null
kpis_met_80,bigint,null
no_of_trainings,double,null
previous_year_rating,double,null
age,double,null
length_of_service,double,null


In [0]:
sdf.write.mode("overwrite").format("parquet").partitionBy("reference_date").save("dbfs:/user/hive/warehouse/hr_analytics/")

In [0]:
%fs
ls user/hive/warehouse/hr_analytics/

path,name,size,modificationTime
dbfs:/user/hive/warehouse/hr_analytics/_SUCCESS,_SUCCESS,0,1671287342000
dbfs:/user/hive/warehouse/hr_analytics/_committed_5426987945020557154,_committed_5426987945020557154,35,1671287341000
dbfs:/user/hive/warehouse/hr_analytics/reference_date=2022-01/,reference_date=2022-01/,0,1671287343000
dbfs:/user/hive/warehouse/hr_analytics/reference_date=2022-02/,reference_date=2022-02/,0,1671287342000
dbfs:/user/hive/warehouse/hr_analytics/reference_date=2022-03/,reference_date=2022-03/,0,1671287343000
dbfs:/user/hive/warehouse/hr_analytics/reference_date=2022-04/,reference_date=2022-04/,0,1671287342000
dbfs:/user/hive/warehouse/hr_analytics/reference_date=2022-05/,reference_date=2022-05/,0,1671287342000
dbfs:/user/hive/warehouse/hr_analytics/reference_date=2022-06/,reference_date=2022-06/,0,1671287343000
dbfs:/user/hive/warehouse/hr_analytics/reference_date=2022-07/,reference_date=2022-07/,0,1671287343000
dbfs:/user/hive/warehouse/hr_analytics/reference_date=2022-08/,reference_date=2022-08/,0,1671287343000


In [0]:
%sql
CREATE TABLE fiep.dataprep_df_parquet USING PARQUET LOCATION 'dbfs:/user/hive/warehouse/hr_analytics/*';

In [0]:
%sql
select * from fiep.dataprep_df_parquet limit 20;

department,region,education,gender,recruitment_channel,kpis_met_80,no_of_trainings,previous_year_rating,age,length_of_service,avg_training_score,employee_id,is_promoted,dataset
Operations,region_7,Below Secondary,f,other,0,1.0,3.329255780917055,26.0,1.0,59.0,23285,0,train
Operations,region_15,Bachelor's,m,other,0,1.0,5.0,46.0,3.0,62.0,37909,0,train
Sales & Marketing,region_2,Missing,m,other,0,1.0,1.0,27.0,1.0,49.0,73951,0,train
Operations,region_15,Bachelor's,m,sourcing,1,1.0,4.0,43.0,13.0,64.0,30593,0,train
Procurement,region_7,Bachelor's,m,sourcing,0,3.0,3.0,41.0,11.0,70.0,28996,0,train
Finance,region_22,Bachelor's,m,other,0,1.0,2.0,30.0,3.0,58.0,38356,0,train
Operations,region_7,Below Secondary,m,other,0,1.0,3.0,24.0,2.0,59.0,64865,0,train
Technology,region_2,Bachelor's,f,sourcing,0,1.0,3.0,29.0,6.0,76.0,59995,0,train
Procurement,region_24,Bachelor's,f,sourcing,0,1.0,2.0,31.0,5.0,73.0,44804,0,train
Operations,region_7,Master's & above,m,other,0,1.0,3.0,33.0,8.0,63.0,3175,0,train


In [0]:
%pip install mlflow hyperopt

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import mlflow
import numpy as np
import pandas as pd
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.ensemble

from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK
from hyperopt.pyll import scope

### Load data
The tutorial uses a dataset describing different wine samples. The [dataset](https://archive.ics.uci.edu/ml/datasets/Wine) is from the UCI Machine Learning Repository and is included in DBFS ([AWS](https://docs.databricks.com/data/databricks-file-system.html)|[Azure](https://docs.microsoft.com/azure/databricks/data/databricks-file-system)|[GCP](https://docs.gcp.databricks.com/data/databricks-file-system.html)).
The goal is to classify red and white wines by their quality. 

For more details on uploading and loading from other data sources, see the documentation on working with data ([AWS](https://docs.databricks.com/data/index.html)|[Azure](https://docs.microsoft.com/azure/databricks/data/index)|[GCP](https://docs.gcp.databricks.com/data/index.html)).

In [0]:
# Load and preprocess data
white_wine = pd.read_csv("/dbfs/databricks-datasets/wine-quality/winequality-white.csv", sep=';')
red_wine = pd.read_csv("/dbfs/databricks-datasets/wine-quality/winequality-red.csv", sep=';')
white_wine['is_red'] = 0.0
red_wine['is_red'] = 1.0
data_df = pd.concat([white_wine, red_wine], axis=0)
data_df.columns = data_df.columns.str.replace(' ', '_')
spark.createDataFrame(data_df).write.mode('overwrite').saveAsTable('fiep.wine')

# Define classification labels based on the wine quality
data_labels = data_df['quality'] >= 7
data_df = data_df.drop(['quality'], axis=1)

# Split 80/20 train-test
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
  data_df,
  data_labels,
  test_size=0.2,
  random_state=1
)

## Part 1. Train a classification model

### MLflow Tracking
[MLflow tracking](https://www.mlflow.org/docs/latest/tracking.html) allows you to organize your machine learning training code, parameters, and models. 

You can enable automatic MLflow tracking by using [*autologging*](https://www.mlflow.org/docs/latest/tracking.html#automatic-logging).

In [0]:
# Enable MLflow autologging for this notebook
mlflow.autolog()

2022/12/17 14:29:16 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/12/17 14:29:16 INFO mlflow._spark_autologging: Autologging successfully enabled for spark.
2022/12/17 14:29:16 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.
2022/12/17 14:29:16 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.


Next, train a classifier within the context of an MLflow run, which automatically logs the trained model and many associated metrics and parameters. 

You can supplement the logging with additional metrics such as the model's AUC score on the test dataset.

In [0]:
with mlflow.start_run(run_name='gradient_boost') as run:
  model = sklearn.ensemble.GradientBoostingClassifier(random_state=0)
  
  # Models, parameters, and training metrics are tracked automatically
  model.fit(X_train, y_train)

  predicted_probs = model.predict_proba(X_test)
  roc_auc = sklearn.metrics.roc_auc_score(y_test, predicted_probs[:,1])
  
  # The AUC score on test data is not automatically logged, so log it manually
  mlflow.log_metric("test_auc", roc_auc)
  print("Test AUC of: {}".format(roc_auc))

Test AUC of: 0.8834365701533531


If you aren't happy with the performance of this model, train another model with different hyperparameters.

In [0]:
# Start a new run and assign a run_name for future reference
with mlflow.start_run(run_name='gradient_boost') as run:
  model_2 = sklearn.ensemble.GradientBoostingClassifier(
    random_state=0, 
    
    # Try a new parameter setting for n_estimators
    n_estimators=300,
  )
  model_2.fit(X_train, y_train)

  predicted_probs = model_2.predict_proba(X_test)
  roc_auc = sklearn.metrics.roc_auc_score(y_test, predicted_probs[:,1])
  mlflow.log_metric("test_auc", roc_auc)
  print("Test AUC of: {}".format(roc_auc))

Test AUC of: 0.8943229285877775


### View MLflow runs
To view the logged training runs, click the **Experiment** icon at the upper right of the notebook to display the experiment sidebar. If necessary, click the refresh icon to fetch and monitor the latest runs. 

<img width="350" src="https://docs.databricks.com/_static/images/mlflow/quickstart/experiment-sidebar-icons.png"/>

You can then click the experiment page icon to display the more detailed MLflow experiment page ([AWS](https://docs.databricks.com/applications/mlflow/tracking.html#notebook-experiments)|[Azure](https://docs.microsoft.com/azure/databricks/applications/mlflow/tracking#notebook-experiments)|[GCP](https://docs.gcp.databricks.com/applications/mlflow/tracking.html#notebook-experiments)). This page allows you to compare runs and view details for specific runs.

<img width="800" src="https://docs.databricks.com/_static/images/mlflow/quickstart/compare-runs.png"/>

### Load models
You can also access the results for a specific run using the MLflow API. The code in the following cell illustrates how to load the model trained in a given MLflow run and use it to make predictions. You can also find code snippets for loading specific models on the MLflow run page ([AWS](https://docs.databricks.com/applications/mlflow/tracking.html#view-notebook-experiment)|[Azure](https://docs.microsoft.com/azure/databricks/applications/mlflow/tracking#view-notebook-experiment)|[GCP](https://docs.gcp.databricks.com/applications/mlflow/tracking.html#view-notebook-experiment)).

In [0]:
# After a model has been logged, you can load it in different notebooks or jobs
# mlflow.pyfunc.load_model makes model prediction available under a common API
model_loaded = mlflow.pyfunc.load_model(
  'runs:/{run_id}/model'.format(
    run_id=run.info.run_id
  )
)

predictions_loaded = model_loaded.predict(X_test)
predictions_original = model_2.predict(X_test)

# The loaded model should match the original
assert(np.array_equal(predictions_loaded, predictions_original))

## Part 2. Hyperparameter Tuning
At this point, you have trained a simple model and used the MLflow tracking service to organize your work. This section covers how to perform more sophisticated tuning using Hyperopt.

### Parallel training with Hyperopt and SparkTrials
[Hyperopt](http://hyperopt.github.io/hyperopt/) is a Python library for hyperparameter tuning. For more information about using Hyperopt in Databricks, see the documentation ([AWS](https://docs.databricks.com/applications/machine-learning/automl-hyperparam-tuning/index.html#hyperparameter-tuning-with-hyperopt)|[Azure](https://docs.microsoft.com/azure/databricks/applications/machine-learning/automl-hyperparam-tuning/index#hyperparameter-tuning-with-hyperopt)|[GCP](https://docs.gcp.databricks.com/applications/machine-learning/automl-hyperparam-tuning/index.html#hyperparameter-tuning-with-hyperopt)).

You can use Hyperopt with SparkTrials to run hyperparameter sweeps and train multiple models in parallel. This reduces the time required to optimize model performance. MLflow tracking is integrated with Hyperopt to automatically log models and parameters.

In [0]:
# Define the search space to explore
search_space = {
  'n_estimators': scope.int(hp.quniform('n_estimators', 20, 1000, 1)),
  'learning_rate': hp.loguniform('learning_rate', -3, 0),
  'max_depth': scope.int(hp.quniform('max_depth', 2, 5, 1)),
}

def train_model(params):
  # Enable autologging on each worker
  mlflow.autolog()
  with mlflow.start_run(nested=True):
    model_hp = sklearn.ensemble.GradientBoostingClassifier(
      random_state=0,
      **params
    )
    model_hp.fit(X_train, y_train)
    predicted_probs = model_hp.predict_proba(X_test)
    # Tune based on the test AUC
    # In production settings, you could use a separate validation set instead
    roc_auc = sklearn.metrics.roc_auc_score(y_test, predicted_probs[:,1])
    mlflow.log_metric('test_auc', roc_auc)
    
    # Set the loss to -1*auc_score so fmin maximizes the auc_score
    return {'status': STATUS_OK, 'loss': -1*roc_auc}

# SparkTrials distributes the tuning using Spark workers
# Greater parallelism speeds processing, but each hyperparameter trial has less information from other trials
# On smaller clusters or Databricks Community Edition try setting parallelism=2
spark_trials = SparkTrials(
  parallelism=8
)

with mlflow.start_run(run_name='gb_hyperopt') as run:
  # Use hyperopt to find the parameters yielding the highest AUC
  best_params = fmin(
    fn=train_model, 
    space=search_space, 
    algo=tpe.suggest, 
    max_evals=32,
    trials=spark_trials)

 16%|█▌        | 5/32 [00:20<01:22,  3.06s/trial, best loss: -0.8983498798352026]

### Search runs to retrieve the best model
Because all of the runs are tracked by MLflow, you can retrieve the metrics and parameters for the best run using the MLflow search runs API to find the tuning run with the highest test auc.

This tuned model should perform better than the simpler models trained in Part 1.

In [0]:
# Sort runs by their test auc; in case of ties, use the most recent run
best_run = mlflow.search_runs(
  order_by=['metrics.test_auc DESC', 'start_time DESC'],
  max_results=10,
).iloc[0]
print('Best Run')
print('AUC: {}'.format(best_run["metrics.test_auc"]))
print('Num Estimators: {}'.format(best_run["params.n_estimators"]))
print('Max Depth: {}'.format(best_run["params.max_depth"]))
print('Learning Rate: {}'.format(best_run["params.learning_rate"]))

best_model_pyfunc = mlflow.pyfunc.load_model(
  'runs:/{run_id}/model'.format(
    run_id=best_run.run_id
  )
)
best_model_predictions = best_model_pyfunc.predict(X_test[:5])
print("Test Predictions: {}".format(best_model_predictions))

### Compare multiple runs in the UI
As in Part 1, you can view and compare the runs in the MLflow experiment details page, accessible via the external link icon at the top of the **Experiment** sidebar. 

On the experiment details page, click the "+" icon to expand the parent run, then select all runs except the parent, and click **Compare**. You can visualize the different runs using a parallel coordinates plot, which shows the impact of different parameter values on a metric. 

<img width="800" src="https://docs.databricks.com/_static/images/mlflow/quickstart/parallel-plot.png"/>